In [1]:
import os
os.environ['OMP_NUM_THREADS']='2'
os.environ['LD_LIBRARY_PATH']=''
os.environ['CUDA_LAUNCH_BLOCKING']='1'

In [2]:
%cd /home/pengyu.zhang/project/superres/ProbSR/Experiment3

/home/pengyu.zhang/project/superres/ProbSR/Experiment3


In [3]:
from data_generation import *
from scipy.linalg import sqrtm
from downscaling import *
from utils import *
import random
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torch.nn.functional as F

## Generate Training Data

In [5]:
N_low = 15
N_high = 60
scale = 4
    
for i in tqdm(range(80)):
    a = random.uniform(-4,4)
    b = random.uniform(-4,4)
    c = random.uniform(-4,4)
    d = random.uniform(-2,2)
    w_low = generate_data(N_low,a,b,c,d)[0]
    w_high = generate_data(N_high,a,b,c,d)[0]
    w_high_tensor = torch.tensor(w_high).to(torch.float32)
    w_low_tensor = torch.tensor(w_low).to(torch.float32)
    downscaled = F.interpolate(w_high_tensor.reshape(1,1,N_high,N_high,N_high),(N_low,N_low,N_low)).reshape(N_low,N_low,N_low)
    residual = (w_low_tensor-downscaled).numpy()
    
    if i == 0:
        total_low = w_low.reshape(1,N_low,N_low,N_low)
        total_high = w_high.reshape(1,N_high,N_high,N_high)
        total_residual = residual.reshape(1,N_low,N_low,N_low)
    else:
        total_low = np.concatenate([total_low,w_low.reshape(1,N_low,N_low,N_low)],axis=0)
        total_high = np.concatenate([total_high,w_high.reshape(1,N_high,N_high,N_high)],axis=0)
        total_residual = np.concatenate([total_residual,residual.reshape(1,N_low,N_low,N_low)],axis=0)

100%|██████████| 80/80 [5:57:42<00:00, 268.28s/it]  


In [7]:
with h5py.File("data/DownBy4_15_60_bicubic.h5", 'w') as hf:
    hf.create_dataset("high_res",  data=total_high)
    hf.create_dataset("low_res",  data=total_low)
    hf.create_dataset("residual", data=total_residual)

In [7]:
create_forcing_term(20,8,3,3).shape

(400,)

In [5]:
N_low = 16
N_high = 64
scale = 4

# Code downscaling matrix
'''H = np.zeros((N_low*N_low, N_high*N_high))

submatrix = np.zeros((N_low,N_high))
for i in range(N_low):
    submatrix[i,scale*i] = 1
    
for j in range(N_low):
    H[N_low*j:N_low*(j+1),N_high*scale*j:N_high*(scale*j+1)] = submatrix'''
    
for i in tqdm(range(1000)):
    a = random.uniform(-4,4)
    b = random.uniform(-4,4)
    c = random.uniform(-4,4)
    d = random.uniform(-2,2)
    w_low = generate_data(N_low,a,b,c,d)[0]
    b_high = create_forcing_term(N_high,a,b,c,d)
    
    if i==0:
        total_low = w_low.reshape(1,N_low,N_low,N_low)
        total_forcing = b_high.reshape(1,N_high**3,1)
    else:
        total_low = np.concatenate([total_low,w_low.reshape(1,N_low,N_low,N_low)],axis=0)
        total_forcing = np.concatenate([total_forcing,b_high.reshape(1,N_high**3,1)],axis=0)

100%|██████████| 1000/1000 [18:44<00:00,  1.12s/it]


In [5]:
total_low.shape

(1000, 15, 15, 15)

In [7]:
with h5py.File("data/16_64_low_forcing.h5", 'w') as hf:
    hf.create_dataset("low_res",  data=total_low)
    hf.create_dataset("forcing", data=total_forcing)